## EDA

In [4]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [7]:
description.head(3)

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False


### Read all the data into a dictionary of DataFrames

In [17]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

# EDA

In [19]:
dfs

{'actor':      actor_id first_name     last_name         last_update
 0         1.0   PENELOPE       GUINESS 2021-03-06 15:51:59
 1         2.0       NICK      WAHLBERG 2021-03-06 15:51:59
 2         3.0         ED         CHASE 2021-03-06 15:51:59
 3         4.0   JENNIFER         DAVIS 2021-03-06 15:51:59
 4         5.0     JOHNNY  LOLLOBRIGIDA 2021-03-06 15:51:59
 ..        ...        ...           ...                 ...
 195     196.0       BELA        WALKEN 2021-03-06 15:52:00
 196     197.0      REESE          WEST 2021-03-06 15:52:00
 197     198.0       MARY        KEITEL 2021-03-06 15:52:00
 198     199.0      JULIA       FAWCETT 2021-03-06 15:52:00
 199     200.0      THORA        TEMPLE 2021-03-06 15:52:00
 
 [200 rows x 4 columns],
 'address':      address_id                   address address2 district  city_id  \
 0             1         47 MySakila Drive     None               300   
 1             2        28 MySQL Boulevard     None               576   
 2            